In [ ]:
import re
import sys
import spacy
import tweepy
import datetime
import xlsxwriter
import wikipedia
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from geotext import GeoText
from nltk.corpus import stopwords    
from nltk.tokenize import word_tokenize 

In [ ]:
citylist=["Bradford" ,"London" ,"Manchester"]
final_dataframe4 = pd.DataFrame(columns=citylist)
nlp = spacy.load('en_core_web_sm')
s2 = "See also"

In [ ]:
len(citylist)

In [ ]:
#def parse_words_first_level():
city_word=[]
allcity_word=[]
city_wikicontent=[]
for city in citylist:
    words_refined_citywise=[]
    print("**********************************************************************")
    print("taking content for", city)
    p=wikipedia.page(city)
    city_wikicontent = p.content
    s1 = []
    string = (city_wikicontent[:city_wikicontent.index(s2)])
    print(string)
    doc = nlp(string)
    print("$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$")
    print("taking each city words thro nlp")
    eachcity_word=[]
    for ent in doc.ents:
        if ent.label_ not in {'QUANTITY', 'DATE', 'ORDINAL', 'CARDINAL', 'PERCENT', 'MONEY'}:
            if ent.text != '\n' and len(ent.text)>2:
                print("appending words" , ent.text)
                eachcity_word.append(ent.text)
    city_unique_words=set(eachcity_word)
    words_refined_citywise=list(city_unique_words)
    print("length of ",city," is", len(words_refined_citywise))
    print(words_refined_citywise)
    print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
    allcity_word.append((city,words_refined_citywise))
    print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
    print("^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^")
    print("length of all city list is", len(allcity_word))
    print(allcity_word)

In [ ]:
allcity_word

In [ ]:
G1 = nx.DiGraph()
G2 = nx.DiGraph()
G3 = nx.DiGraph()
G = [G1,G2,G3]

In [ ]:
combine_word = zip(G,citylist,allcity_word)

In [ ]:
combine_words=list(combine_word)

In [ ]:
combine_words[0]

In [ ]:
count = 0
counter_list=[]
complete_pack=[]
eachcity_word2=[]
city_words_links=[]
cities_link_content=[]
each_city_link=[]
for x in combine_words:
    citylinks = (wikipedia.page(x[count+1])).links
    each_city_link.append((x[count+1],citylinks))
    obj=x[0]
    obj.add_node(x[count+1])
    #print(x[count+1])
    for y in citylinks:
        city_link_content=[]
        print(y)
        obj.add_node(y)
        obj.add_edge(x[count+1],y)
        words_wiki = wikipedia.page(y)
        wiki_cont = words_wiki.content
        words_wikilinks = words_wiki.links
        s1 = []
        doc = []
        counter = 0
        # i think i can do removal of stop words first
        if s2 in wiki_cont:
            string = (wiki_cont[:wiki_cont.index(s2)])
            doc = nlp(string)
        else:
            doc = nlp(wiki_cont)
        for ent in doc.ents:
                if ent.label_ not in {'QUANTITY', 'DATE', 'ORDINAL', 'CARDINAL', 'PERCENT', 'MONEY'}:
                    if ent.text != '\n' and len(ent.text)>2:
                        city_link_content.append(ent.text)
        removing_duplicates = list(set(city_link_content))
        cities_link_content.append(removing_duplicates)
        #---- implement -- cross check with all cities and see how much they match
        for h in removing_duplicates:
            word = h
            if word in x[2][1]:
                counter+=1
        counter_list.append((x[1],y,counter,len(x[2][1]),len(city_link_content)))
        for z in words_wikilinks:
            if z in citylinks:
                obj.add_edge(y,x[count+1])
            else:
                obj.add_node(z)
                obj.add_edge(y,z)
        city_words_links.append((x[1],y,removing_duplicates))

In [ ]:
each_city_link

In [ ]:
nx.draw_networkx(G1)

plt.show()

In [ ]:
city_words_links[5]

In [ ]:
pagerank_cities=[]
for graph in G:
    pr = nx.pagerank(graph, alpha=0.9)
    pagerank_cities.append((pr))

In [ ]:
print(pagerank_cities)

In [ ]:
 counter_list

In [ ]:
# Similarity measures by overlapping:
similarity_overlaping=[]
for g in counter_list:
    static = g[2]
    similarity = static/(g[3]+g[4])
    similarity_overlaping.append((g[0],g[1],similarity))
print(similarity_overlaping)

df3 = pd.DataFrame(columns=['city', 'link', 'similarity'])
for i in range (len(similarity_overlaping)):
    df3.at[i, 'city'] = similarity_overlaping[i][0]
    df3.at[i, 'link'] = similarity_overlaping[i][1]
    df3.at[i, 'similarity'] = similarity_overlaping[i][2]

In [ ]:
each_city_link[0]

# Taking top 3 cities using overlapping scores:

In [ ]:
from collections import Counter

In [ ]:
def cal_score_overlap(ip_words):
    d1 = {}
    d3={}
    tweet = ip_words
    calculating_score1=[]
    calculating_score2=[]
    calculating_score3=[]
    for s in tweet:
        print(s)
        for o in city_words_links:
            if s in o[2]:
                score=float(df3.loc[df3['link'] == o[1], 'similarity'])
                calculating_score1.append((o[0],score))
            else:
                calculating_score1.append((o[0],0))
        for word in allcity_word:
            #print(word[1])
            if s in word[1]:
                calculating_score2.append((word[0],0.5))
            else:
                calculating_score2.append((word[0],0))
        for ci in each_city_link:
            if s in ci[1]:
                calculating_score3.append((ci[0],0.75))
            else:
                calculating_score3.append((ci[0],0))
    for company, n in calculating_score1:
        key = (company)
        d1[key] = d1.get(key, 0) + n
    #print(d1)
    for company3, n3 in calculating_score3:
        key3 = (company3)
        d3[key3] = d3.get(key3, 0) + n3
    #print(d3)
    d2 = {}
    for company2, n2 in calculating_score2:
        key2 = (company2)
        d2[key2] = d2.get(key2, 0) + n2
    #print(d2)
    final_score = { k: d1.get(k, 0) + d2.get(k, 0) + d2.get(k, 0)  for k in set(d1) | set(d2) | set(d3)}
    print('final',final_score)
    return final_score

In [ ]:
df3

# Taking top 3 cities using betweeness centrality

In [ ]:
sp = nx.algorithms.centrality.betweenness_centrality(G1,normalized=True)

In [ ]:
sp

# Extracting more tweets of an user

In [ ]:
# handling rate limit errors
def limit_handled(cursor):
    while True:
        try:
            yield cursor.next()
        except tweepy.RateLimitError:
            print("Time Limit. Sleeping now....")
            time.sleep(15 * 60)
            print("Trying again")

In [ ]:
def extractTweets(screen_name):
    #create empty data frame
    dataFrame = pd.DataFrame()
    tweet_count=0
    retweet_count=0
    fav_count=0
    all_words =[]
    tweet_text=[] # list of tweets
    retweet_text=[] # list of retweets
    fav_tweet_text=[] # list of favorite tweets
    all_geo_text=[]
    for tweets in limit_handled(tweepy.Cursor(api.user_timeline,screen_name=screen_name,count=200,tweet_mode='extended').items()):
        #print(tweets)
        #print("Created at:%s, tweet is : %s " %(tweets.created_at,tweets.text))
        if (tweets.lang == "en"):
            raw_text = tweets.full_text
            text = re.sub(r"http\S+","",raw_text)
            geo_text = GeoText(removeNonAscii(text)).cities
            all_geo_text+=geo_text
            twt_text = (nlp(removeNonAscii(text)))
            for ent in twt_text.ents:
                #print('twt_text',ent)
                if ent.label_ not in {'QUANTITY', 'DATE', 'ORDINAL', 'CARDINAL', 'PERCENT', 'MONEY'}:
                    if ent.text != '\n' and len(ent.text)>2:
                        print("appending words" , ent.text)
                        tweet_text.append(ent.text)
            tweet_count+=1
        if hasattr(tweets,'retweeted_status'):
            if (tweets.retweeted_status.lang == "en"):
                raw_text = tweets.retweeted_status.full_text
                text = re.sub(r"http\S+","",raw_text)
                retwt_text = (nlp(removeNonAscii(text)))
                geo_text1 = GeoText(removeNonAscii(text)).cities
                all_geo_text+=geo_text1
                for ent in retwt_text.ents:
                    #print('retweet text',ent)
                    if ent.label_ not in {'QUANTITY', 'DATE', 'ORDINAL', 'CARDINAL', 'PERCENT', 'MONEY'}:
                        if ent.text != '\n' and len(ent.text)>2:
                            print("appending words" , ent.text)
                            retweet_text.append(ent.text)
                retweet_count+=1        
        if(tweet_count == 1000 or retweet_count == 1000):
            break
    for fav_tweets in limit_handled(tweepy.Cursor(api.favorites,id=screen_name,tweet_mode='extended').items()):
        if(fav_tweets.lang  == 'en'):
            raw_text = fav_tweets.full_text
            text = re.sub(r"http\S+","",raw_text)
            geo_text2 = GeoText(removeNonAscii(text)).cities
            all_geo_text+=geo_text2
            fav_text = (nlp(removeNonAscii(text)))
            for ent in fav_text.ents:
                #print('retweet text',ent)
                if ent.label_ not in {'QUANTITY', 'DATE', 'ORDINAL', 'CARDINAL', 'PERCENT', 'MONEY'}:
                    if ent.text != '\n' and len(ent.text)>2:
                        #print("appending words" , ent.text)
                        fav_tweet_text.append(ent.text)
                        
            fav_count+=1
        if (fav_count > 50):
            break
    all_words = tweet_text+retweet_text+fav_tweet_text+all_geo_text
    #print('all_words',all_words)
    return all_words

In [ ]:
def authentication():
    auth = tweepy.OAuthHandler('gcVOEVJqRYvglu1k5KzsdEqMM','ikxcyneBliQZm0Ea8jDV7SX3myuLoctt6vjSDYhL2uDr5hYmqU')
    auth.set_access_token('972247239824461826-tdmBijOQHifuQ5WOJBMkU9rUd8ThAMy','xHaN3RZz48JDHqnIQVxR15DdeHnb7OQEjQCtPZSWar250')
    api=tweepy.API(auth,wait_on_rate_limit=True)
    return api

In [ ]:
api = authentication()

In [ ]:
def removeNonAscii(text):
    return "".join(char for char in text if ord(char) < 128)

In [ ]:
df=pd.read_excel('Brad.xlsx')

In [ ]:
df_en = pd.DataFrame(df.loc[df['Tweet language (ISO 639-1)'] == 'en'])

In [ ]:
### change lists to df_en['User Name'] while submitting
for ing in range(len(df_en[:1500])):
    print(df.loc[df.index[ing],'Tweet Id'])
    #words_From_prev_tweets = extractTweets(screenname)
    #print('afterrrrrrrrrr extractTweets_applynlp')
    words_bio_data=[]
    geo_text_here=[]
    bio=df.loc[df.index[ing],'Bio']
    #print('biooooooooooo',bio)
    id_of_tweet=df.loc[df.index[ing],'Tweet Id']
    #print(id_of_tweet)
    try:
        tweetz = api.get_status(id_of_tweet)
        tweet=tweetz.text
    except:
        print('tweet deleted')
        tweet=df.loc[df.index[ing],'Tweet content']
    text1 = re.sub(r"http\S+","",str(bio))
    text2 = re.sub(r"http\S+","",str(tweet))
    doc3 = nlp(removeNonAscii(text1))
    doc4 = nlp(removeNonAscii(text2))    
    #print('doc3',doc3, 'doc44',doc4)
    geo_text_here+=GeoText(removeNonAscii(text1)).cities
    geo_text_here+=GeoText(removeNonAscii(text2)).cities
    #print('getting geo text both in tweet and bio')
    #print(geo_text_here)
    for ent in doc3.ents:
        if ent.label_ not in {'QUANTITY', 'DATE', 'ORDINAL', 'CARDINAL', 'PERCENT', 'MONEY'}:
            if ent.text != '\n' and len(ent.text)>2:
                words_bio_data.append(ent.text)
    for ent in doc4.ents:
        if ent.label_ not in {'QUANTITY', 'DATE', 'ORDINAL', 'CARDINAL', 'PERCENT', 'MONEY'}:
            if ent.text != '\n' and len(ent.text)>2:
                words_bio_data.append(ent.text)
    non_duplicate_words_here = list(set(words_bio_data))
    print('non_duplicate_words_here',non_duplicate_words_here)
    #print('words_From_prev_tweets',words_From_prev_tweets)
    #print(list(set(geo_text_here)))
    #final_list = non_duplicate_words_here + words_From_prev_tweets + list(set(geo_text_here))
    final_list = non_duplicate_words_here + list(set(geo_text_here))

    print('finallist for',final_list)
    filldataframe = cal_score_overlap(final_list)
    print(filldataframe)
    final_dataframe4 = final_dataframe4.append(filldataframe,ignore_index=True)

In [ ]:
lat=df_en['Latitude']

In [ ]:
long=df_en['Longitude']

In [ ]:
final_dataframe4['latitude'] = lat
final_dataframe4['longitude'] = long

In [ ]:
datamodel

In [ ]:
# Finding Latitude
datamodel = final_dataframe4.dropna()
lat_values= datamodel['latitude']
long_values=datamodel['longitude']
datamodel_ip = datamodel.drop(['latitude','longitude'],axis=1)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(datamodel_ip, lat_values, test_size=0.30, random_state=42)

In [ ]:
from sklearn.model_selection import train_test_split
X_train1, X_test1, y_train1, y_test1 = train_test_split(datamodel_ip, long_values, test_size=0.30, random_state=42)

In [ ]:
from sklearn import svm 
model = svm.SVR(kernel='linear',C=1, gamma=1) 
# there is various option associated with it, like changing kernel, gamma and C value. Will discuss more # about it in next section.Train the model using the training sets and check score
model.fit(X_train, y_train)
predicted= model.predict(X_test)

In [ ]:
model1 = svm.SVR(kernel='linear',C=1, gamma=1) 
model1.fit(X_train1, y_train1)
predicted1= model1.predict(X_test1)

In [ ]:
accuracy_df = pd.DataFrame()
accuracy_df['Actual Lat'] = y_test
accuracy_df['Actal Long'] = y_test1
accuracy_df['Predicted Lat'] = predicted
accuracy_df['Predicted Long'] = predicted1

In [ ]:
from math import sin, cos, sqrt, atan2, radians
over150=[]
below150=[]

In [ ]:
list1 = list(zip(y_test,y_test1))
list2 = list(zip(predicted,predicted1))
for i in range(len(list1)):
    lat1 = list1[i][0]
    lon1 = list1[i][1]
    lat2 = list2[i][0]
    lon2 = list2[i][1]
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = R * c
    if distance>150:
        over150.append(distance)
    else:
        below150.append(distance)
    print("Result:", distance)
print('Accuracy' ,(len(below150)/(len(over150)+len(below150)))*100,'%')